The following cell holds the definition of our parameters,
these values can be overriden by rendering the with e.g. the following command:

```bash
papermill -p alpha 0.2 -p ratio 0.3 universe_analysis.ipynb output/test_run.ipynb
```


In [4]:
run_no = 0
universe_id = "test"
universe = {
    "scale": "scale", # "scale", "do-not-scale",
    "encode_categorical": "one-hot", # "ordinal", "one-hot"
    "stratify_split": "none", # "none", "target", "protected-attribute", "both",
    "model": "elasticnet", # "logreg", "rf", "svm", "gbm", "elasticnet"
    "cutoff": ["raw_0.5", "quantile_0.1", "quantile_0.25"],
    "preprocess_age": "quantiles_3", # "none", "bins_10", "quantiles_3", "quantiles_4"
    "preprocess_income": "bins_10000", # "none", "log", "bins_10000", "quantiles_3", "quantiles_4"
    "exclude_features": "none", # "race", "sex", "race-sex"
    "exclude_subgroups": "drop-name_race_Some Other Race alone", # keep-all, drop-smallest_race_2, keep-largest_race_1, keep-largest_race_2, drop-name_race_Some Other Race alone
    "eval_fairness_grouping": ["majority-minority", "race-all"],
    "eval_exclude_subgroups": ["exclude-in-eval", "keep-in-eval"],
    "eval_on_subset": [
        "full",
        # Largest PUMA region
        "locality-largest-only",
        # PUMA region w/ highest share of white people
        "locality-whitest-only",
        # PUMA regions belonging to a large city
        "locality-city-la",
        "locality-city-sf",
        # Exclude military personnel from test dataset
        "exclude-military",
        # Exclude non US citizens from test dataset
        "exclude-non-citizens",
    ],
}
output_dir="./output"
seed=0

In [5]:
import json
# Parse universe into dict if it is passed as a string
if isinstance(universe, str):
    universe = json.loads(universe)

In [6]:
# Auto-reload the custom package
%load_ext autoreload
%autoreload 1
%aimport fairness_multiverse


In [7]:
from fairness_multiverse.universe import UniverseAnalysis

universe_analysis = UniverseAnalysis(
    run_no = run_no,
    universe_id = universe_id,
    universe = universe,
    output_dir=output_dir,
)

Always use the same seed


In [8]:
import numpy as np
parsed_seed = int(seed)
np.random.seed(parsed_seed)
print(f"Using Seed: {parsed_seed}")

Using Seed: 0


## Loading Data

Using some newer features of the `folktables` package. Therefore it needs to be installed from github directly. Also using a forked version of the package to implement a fix faster (install via `pip install --upgrade --force-reinstall git+https://github.com/jansim/folktables`).

### (Down)load Data from Census


In [12]:
from pathlib import Path
import pandas as pd
from folktables import ACSDataSource

data_source = ACSDataSource(
    survey_year='2018',
    horizon='1-Year',
    survey='person'
)

# Use custom caching of data
cache_dir = Path("data")
cache_dir.mkdir(exist_ok=True)
cache_file = cache_dir / "dataset.csv.gz"
if cache_file.exists():
    print(f"Loading data from cache: {cache_file}")
    dataset = pd.read_csv(cache_file)
else:
    # Load dataset via folktables, if necessary download from the internet
    dataset = data_source.get_data(states=["CA"], download=True)
    # Write file to cache
    dataset.to_csv(cache_file, index=False)

Loading data from cache: data/dataset.csv.gz


In [14]:
dataset

,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
0,P,2018GQ0000004,9,1,3701,4,6,1013097,32,30,...,34,60,60,7,8,59,33,8,58,32
1,P,2018GQ0000013,9,1,7306,4,6,1013097,45,18,...,0,0,0,91,46,46,0,89,45,0
2,P,2018GQ0000016,9,1,3755,4,6,1013097,109,69,...,105,232,226,110,114,217,2,111,2,106
3,P,2018GQ0000020,9,1,7319,4,6,1013097,34,25,...,67,0,34,34,69,0,34,35,0,0
4,P,2018GQ0000027,9,1,6511,4,6,1013097,46,31,...,47,81,10,11,79,47,44,81,47,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378812,P,2018HU1400891,9,2,1308,4,6,1013097,94,61,...,26,77,21,171,86,31,25,82,82,106
378813,P,2018HU1400893,9,1,7108,4,6,1013097,172,40,...,173,53,288,47,199,181,164,178,176,154
378814,P,2018HU1400893,9,2,7108,4,6,1013097,156,43,...,142,51,259,46,171,150,152,169,165,148
378815,P,2018HU1400893,9,3,7108,4,6,1013097,172,23,...,178,56,297,56,189,167,167,177,180,155


In [ ]:
# Download additional definition data
definition_df = data_source.get_definitions(download=True)

### Perform Pre-Processing for Selected Task

- **ACSIncome**: predict whether an individual’s income is above $50,000, after filtering the ACS PUMS data sample to only include individuals above the age of 16, who reported usual working hours of at least 1 hour per week in the past year, and an income of at least $100. The threshold of $50,000 was chosen so that this dataset can serve as a replacement to UCI Adult, but we also offer datasets with other income cutoffs described in Appendix B.
- **ACSPublicCoverage**: predict whether an individual is covered by public health insurance, after filtering the ACS PUMS data sample to only include individuals under the age of 65, and those with an income of less than $30,000. This filtering focuses the prediction problem on low-income individuals who are not eligible for Medicare.
- **ACSMobility**: predict whether an individual had the same residential address one year ago, after filtering the ACS PUMS data sample to only include individuals between the ages of 18 and 35. This filtering increases the difficulty of the prediction task, as the base rate of staying at the same address is above 90% for the general population.
- **ACSEmployment**: predict whether an individual is employed, after filtering the ACS PUMS data sample to only include individuals between the ages of 16 and 90.
- **ACSTravelTime**: predict whether an individual has a commute to work that is longer than 20 minutes, after filtering the ACS PUMS data sample to only include individuals who are employed and above the age of 16. The threshold of 20 minutes was chosen as it is the US-wide median travel time to work in the 2018 ACS PUMS data release

- The selected story & task has implications for which fairness metric makes the most sense in the end.


In [ ]:
from copy import deepcopy
from folktables import generate_categories
from folktables import ACSPublicCoverage

# Normally you would create the task with the following snippet
# features, label, group = ACSEmployment.df_to_numpy(acs_data)
# But this severly limits us in regards to how many protected
# groups we can examine and further removes feature lables

task = deepcopy(ACSPublicCoverage)

# Additional features to extract, that are not part of the task
extra_feature_cols = ["PUMA"]
task._features.extend(extra_feature_cols)

categories = generate_categories(features=task.features, definition_df=definition_df)
features_org, label_org, group_org = task.df_to_pandas(dataset, categories=categories)

# Keep a reference to the original state of featuers
features = features_org.copy()

# Immediately remove the extra features before they could leak into the task
features.drop(columns=extra_feature_cols, inplace=True)
extra_features = features_org[extra_feature_cols].copy()

label = label_org.copy()
group = group_org.copy()


In [ ]:
# Let's see the data
features.head(10)

## Preprocessing Data

```
# Invert the categories dictionary
categories_inverted = {
    column: {v: k for k, v in mapping.items()}
    for column, mapping in categories.items()
}
```


### Exclude Protected Features

In [ ]:
# Note this will alwqys be n >= 1, even if empty!
excluded_features = universe["exclude_features"].split("-")
excluded_features_dictionary = {
    "race": "RAC1P",
    "sex": "SEX",
    "immigration": "NATIVITY",
}

# Code nice names to column names
excluded_feature_columns = [
    excluded_features_dictionary[f] for f in excluded_features if len(f) > 0 and f != "none"
]

if len(excluded_feature_columns) > 0:
    print(f"Dropping features: {excluded_feature_columns}")
    features.drop(excluded_feature_columns, axis=1, inplace=True)


### Continuous Variables: Binning / Log-Scaling / Keeping Them As-Is

In [ ]:
from sklearn.pipeline import make_pipeline
from fairness_multiverse.universe import preprocess_continuous

transformer_age, bins_age = preprocess_continuous(source_data=features, column_name="AGEP", configuration=universe["preprocess_age"])
transformer_income, bins_income = preprocess_continuous(source_data=features, column_name="PINCP", configuration=universe["preprocess_income"])

continuous_processor = make_pipeline(
    transformer_age,
    transformer_income
)

### Categorical Variables: One-Hot or Ordinal Encoding

In [ ]:
from sklearn.pipeline import TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

all_categorical_columns = list(set(categories.keys()).intersection(set(features.columns)))

# For which columns is ordinal encoding even an option?
categorical_columns_to_transform = [
    'SCHL',
    # 'MAR',
    # 'SEX',
    # 'DIS',
    # 'ESP',
    # 'CIT',
    # 'MIG',
    'MIL',
    # 'ANC',
    # 'NATIVITY',
    # 'DEAR',
    # 'DEYE',
    # 'DREM',
    # 'ESR',
    # 'ST',
    # 'FER',
    # 'RAC1P'
]

# Support to-be-binned continuous variables
def add_binned_variable_to_categorical_transformation(colname, values):
    if values is not None:
        categorical_columns_to_transform.append(colname)
        categories[colname] = {val: val for val in values}

add_binned_variable_to_categorical_transformation("AGEP", bins_age)
add_binned_variable_to_categorical_transformation("PINCP", bins_income)

def nested_list(all_categories, columns_to_use):
    categories = { col: all_categories[col] for col in columns_to_use }
    # Create a nested list from the categories dict
    categories_list = [[v for k, v in mapping.items()] for column, mapping in categories.items()]
    return categories_list

if (universe["encode_categorical"] == "ordinal"):
    categorical_transformer = OrdinalEncoder(
        categories = nested_list(categories, categorical_columns_to_transform),
    )
elif (universe["encode_categorical"] == "one-hot"):
    categorical_transformer = OneHotEncoder(
        categories = nested_list(categories, categorical_columns_to_transform),
        sparse_output=False
    )
else:
    raise "Unsupported universe option for encode_categorical"

# One-Hot Encode all other cateogircal columns
other_categorical_columns = list(set(all_categorical_columns) - set(categorical_columns_to_transform))
other_transformer = OneHotEncoder(
    categories = nested_list(categories, other_categorical_columns),
    sparse_output=False
)

categorical_preprocessor = ColumnTransformer([
        ("encode_categorical", categorical_transformer, categorical_columns_to_transform),
        ("encode_categorical_rest", other_transformer, other_categorical_columns),
    ],
    remainder='passthrough',
)

## Split Data


In [ ]:
# Select stratification strategy
if universe["stratify_split"] == "none":
    stratify = None
elif universe["stratify_split"] == "target":
    stratify = label
elif universe["stratify_split"] == "protected-attribute":
    stratify = features_org["RAC1P"]
elif universe["stratify_split"] == "both":
    # Concatinate both columns
    stratify = features_org["RAC1P"].astype(str) + "-" + label["PUBCOV"].astype(str)

stratify

In [ ]:
from sklearn.model_selection import train_test_split

(
    X_train, X_test,
    y_train, y_true,
    group_train, group_test,
    org_train, org_test
) = train_test_split(
    features,
    label,
    group,
    features_org,
    test_size=0.2,
    # Note: The analysis originally used two distinct seeds, one for numpy (defaulting to 2023) and one for the train_test_split (defaulting to 0).
    # To allow for exact reproducibility of the original results, as well as specification of only a single seed we base this second seed off the first one.
    # If you adapt this code for your own analysis feel free to remove this line and replace it e.g. with e.g. a call to numpy.random.randint.
    random_state=abs(parsed_seed - 2023),
    stratify=stratify
)

## Post-Splitting Processing

If e.g. only train data is affected

### Exclude Certain Subgroups

In [ ]:
# Extract configuration
exclude_subgroups_config = universe["exclude_subgroups"].split("_")
if len(exclude_subgroups_config) == 1:
    exclude_subgroups_config = (exclude_subgroups_config[0], None, None)
excl_subgroups_method, excl_subgroup_colname, excl_subgroups_value = exclude_subgroups_config

if excl_subgroup_colname == "race":
    excl_subgroup_column = "RAC1P"
    excl_subgroup_counts = org_train[excl_subgroup_column].value_counts()
elif excl_subgroups_method != "keep-all":
    raise Exception("Unsupported configuration for exclude_subgroups:" + universe["exclude_subgroups"])

if excl_subgroups_method == "keep-all":
    # Don't need to do anything
    excl_subgroup_column = None
    excl_subgroup_values = []
else:
    if excl_subgroups_method == "drop-smallest":
        drop_smallest_n = int(excl_subgroups_value)
        excl_subgroup_values = list(excl_subgroup_counts.tail(drop_smallest_n).index)
    elif excl_subgroups_method == "keep-largest":
        keep_largest_n = int(excl_subgroups_value)
        excl_subgroup_values = list(excl_subgroup_counts.tail(
            len(excl_subgroup_counts) - keep_largest_n
        ).index)
    elif excl_subgroups_method == "drop-name":
        excl_subgroup_values = [excl_subgroups_value]
    else:
        raise Exception("Unsupported configuration for exclude_subgroups:" + universe["exclude_subgroups"])

    if excl_subgroup_column is not None:
        print(f"Dropping values: {excl_subgroup_values}")
        keep_rows_mask = ~org_train[excl_subgroup_column].isin(excl_subgroup_values)

    n_rows_to_drop = (~keep_rows_mask).sum()
    if n_rows_to_drop > 0:
        print(f"Dropping N = {n_rows_to_drop} ({n_rows_to_drop / len(keep_rows_mask):.2%}) rows from {excl_subgroup_colname}")
        X_train = X_train[keep_rows_mask]
        y_train = y_train[keep_rows_mask]
        group_train = group_train[keep_rows_mask]

## Fitting the Model

Select which model to fit

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

if (universe["model"] == "logreg"):
    model = LogisticRegression()
elif (universe["model"] == "rf"):
    model = RandomForestClassifier()
elif (universe["model"] == "svm"):
    model = SVC()
elif (universe["model"] == "gbm"):
    model = GradientBoostingClassifier()
elif (universe["model"] == "elasticnet"):
    model = LogisticRegression(penalty = 'elasticnet', solver = 'saga', l1_ratio = 0.5)
else:
    raise "Unsupported universe.model"

In [ ]:
from fairness_multiverse.universe import predict_w_threshold

In [ ]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

model = Pipeline([
    ("continuous_processor", continuous_processor),
    ("categorical_preprocessor", categorical_preprocessor),
    ("scale", StandardScaler() if universe["scale"] == "scale" else None),
    ("model", model),
])

model.fit(X_train, y_train)

y_prob = model.predict_proba(X_test)
y_pred_default = predict_w_threshold(y_prob, 0.5)

# Naive prediction
accuracy_score(y_true = y_true, y_pred = y_pred_default)

In [ ]:
# Evaluate
model.predict(X_test)

## (Fairness) Metrics

- Using [Fairlearn](https://fairlearn.org/v0.8/quickstart.html)


In [ ]:
import numpy as np

colname_to_bin = "RAC1P"
majority_value = features_org[colname_to_bin].mode()[0]

org_test["majmin"] = np.where(org_test[colname_to_bin] == majority_value, "majority", "minority")

The greater of two metrics: `true_positive_rate_difference` and
`false_positive_rate_difference`. The former is the difference between the
largest and smallest of $P[h(X)=1 | A=a, Y=1]$, across all values :math:`a`
of the sensitive feature(s). The latter is defined similarly, but for
$P[h(X)=1 | A=a, Y=0]$.
The equalized odds difference of 0 means that all groups have the same
true positive, true negative, false positive, and false negative rates. [src](https://fairlearn.org/main/api_reference/generated/fairlearn.metrics.equalized_odds_difference.html)

> This shouldn't differ based on which class we see as "good" or "bad"?

In [ ]:
example_universe = universe.copy()
example_universe["cutoff"] = example_universe["cutoff"][0]
example_universe["eval_fairness_grouping"] = example_universe["eval_fairness_grouping"][0]
fairness_dict, metric_frame = universe_analysis.compute_metrics(
    example_universe,
    y_pred_prob=y_prob,
    y_test=y_true,
    org_test=org_test,
)

### Overall

#### Fairness

Main fairness target: **Equalized Odds**.
Seems to be a better fit than equal opportunity, since we're not only interested in Y = 1.
Seems to be a better fit than demographic parity, since we also care about accuracy, not just equal distribution of preds.


Pick column for computation of fairness metrics

#### Performance

Overall performance measures, most interesting in relation to the measures split by group below


In [ ]:
metric_frame.overall

### By Group


In [ ]:
metric_frame.by_group

Graphical comparison:

In [ ]:
# In a graphic
metric_frame.by_group.plot.bar(
    subplots=True,
    layout=[3, 3],
    legend=False,
    figsize=[12, 8],
    title="Show all metrics",
)

### Equality of opportunity violation

```python
white_tpr = np.mean(y_pred[(y_true == 1) & (group_test == 1)])
black_tpr = np.mean(y_pred[(y_true == 1) & (group_test == 2)])

white_tpr - black_tpr
```

## Final Output


In [ ]:
sub_universes = universe_analysis.generate_sub_universes()
len(sub_universes)

In [ ]:
import pandas as pd

# Uses excl_subgroup_column and values from the global scope

def filter_sub_universe_data(sub_universe, org_test):
  # Generate an all True mask to start with
  keep_rows_mask = np.ones(org_test.shape[0], dtype=bool)

  # Potentially remove any subgroups from the test set
  if (sub_universe["eval_exclude_subgroups"] == "exclude-in-eval"):
    if excl_subgroup_column is not None:
      assert excl_subgroup_values is not None

      exclude_subgroup_eval_mask = ~org_test[excl_subgroup_column].isin(excl_subgroup_values)
      keep_rows_mask = keep_rows_mask & exclude_subgroup_eval_mask

      n_rows_to_drop = (~exclude_subgroup_eval_mask).sum()
      print(f"[drop subgroups] Dropping N = {n_rows_to_drop} ({n_rows_to_drop / len(keep_rows_mask):.2%}) rows from {excl_subgroup_colname}")
  elif (sub_universe["eval_exclude_subgroups"] == "keep-in-eval"):
    pass
  else:
    raise "Unsupported eval_exclude_subgroups"

  # Potentially use a smaller and more "convenient" subset of the data to do evalaution on
  if (sub_universe["eval_on_subset"] == "full"):
    pass
  else:
    if sub_universe["eval_on_subset"].startswith("locality"):
      # Filter based on locality / region
      # Step 1: Decide which regions to keep
      if (sub_universe["eval_on_subset"] == "locality-largest-only"):
        # Use the largest PUMA region
        puma_regions_to_keep = [org_test["PUMA"].value_counts().idxmax()]
      elif (sub_universe["eval_on_subset"] == "locality-whitest-only"):
        # Find the majority class on the prot. attribute
        majority_class = org_test["RAC1P"].value_counts().index[0]
        majority_class

        # Find the PUMA region with the highest share of the majority class
        counts = pd.DataFrame()
        counts["full"] = org_test["PUMA"].value_counts(sort=False)
        counts["majority"] = org_test[org_test["RAC1P"] == majority_class]["PUMA"].value_counts(sort=False)
        counts["fraction"] = counts["majority"] / counts["full"]

        # Use the PUMA region with the highest share of the majority class
        majority_puma_id = counts.sort_values(by="fraction", ascending=False).index[0]
        puma_regions_to_keep = [majority_puma_id]
      elif (sub_universe["eval_on_subset"] == "locality-city-la"):
        puma_regions_to_keep = list(range(3701, 3769+1))
      elif (sub_universe["eval_on_subset"] == "locality-city-sf"):
        puma_regions_to_keep = list(range(7501, 7507+1))

      # Step 2: Keep only those regions
      print(f"Keeping the following PUMA regions: {puma_regions_to_keep}")
      eval_on_subset_mask = org_test["PUMA"].isin(puma_regions_to_keep)
    elif (sub_universe["eval_on_subset"] == "exclude-military"):
      # Only keep non-military personnel
      eval_on_subset_mask = (org_test["MIL"].isin(["Never served in the military", "N/A (less than 17 years old)"]))
    elif (sub_universe["eval_on_subset"] == "exclude-non-citizens"):
      # Only keep US citizens
      eval_on_subset_mask = ~(org_test["CIT"] == "Not a citizen of the U.S.")
    else:
      raise "Unsupported eval_on_subset"

    keep_rows_mask = keep_rows_mask & eval_on_subset_mask

    n_rows_to_drop = (~eval_on_subset_mask).sum()
    print(f"[subset] Dropping N = {n_rows_to_drop} ({n_rows_to_drop / len(keep_rows_mask):.2%}) rows")

  n_rows_to_drop = (~keep_rows_mask).sum()
  print(f"[TOTAL] Dropping N = {n_rows_to_drop} ({n_rows_to_drop / len(keep_rows_mask):.2%}) rows. Final size: {keep_rows_mask.sum()}.")

  return keep_rows_mask

In [ ]:
final_output = universe_analysis.generate_final_output(
    y_pred_prob=y_prob,
    y_test=y_true,
    org_test=org_test,
    save=True,
    filter_data=filter_sub_universe_data
)
final_output